In [ ]:
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

In [ ]:
def load_image(image_id):
    image = cv2.imread(image_id)
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# train_images = train_data["image_id"].sample(100, random_state=2020).progress_apply(load_image)

In [ ]:
def init_grabcut_mask(h, w):
    mask = np.ones((h, w), np.uint8) * cv2.GC_PR_BGD
    mask[h//4:3*h//4, w//4:3*w//4] = cv2.GC_PR_FGD
    mask[2*h//5:3*h//5, 2*w//5:3*w//5] = cv2.GC_FGD
    return mask

# plt.imshow(init_grabcut_mask(3*136, 3*205))

In [ ]:
def add_contours(image, mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    if len(contours) != 0:
        cv2.drawContours(image, contours, -1, (255, 0, 0), 3)
        c = max(contours, key = cv2.contourArea)
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0) ,2)
    


def remove_background(image):
    h, w = image.shape[:2]
    mask = init_grabcut_mask(h, w)
    bgm = np.zeros((1, 65), np.float64)
    fgm = np.zeros((1, 65), np.float64)
    cv2.grabCut(image, mask, None, bgm, fgm, 1, cv2.GC_INIT_WITH_MASK)
    mask_binary = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
#     result = cv2.bitwise_and(image, image, mask = mask_binary)
#     add_contours(result, mask_binary) # optional, adds visualizations
    return mask_binary



In [ ]:
pip install imutils

In [ ]:
import imutils

im = load_image("../input/plant-pathology-2021-fgvc8/test_images/ad8770db05586b59.jpg")
im = cv2.resize(im, (150, 100))
mask = remove_background(im)

cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
c = max(cnts, key=cv2.contourArea)
# draw the contours of c

x, y, w, h = cv2.boundingRect(c)
print(x, y, w, h)

crop_img = im[y:y+h, x:x+w]
cv2.drawContours(im, [c], -1, (0, 0, 255), 2)
plt.imshow(crop_img)

In [ ]:
%%time
xy = []
import os
train = os.listdir("../input/plant-pathology-2021-fgvc8/train_images")[:10000]
for i in train:
    im = load_image("../input/plant-pathology-2021-fgvc8/train_images/"+i)
    im = cv2.resize(im, (150, 100))
    mask = remove_background(im)
    
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    # draw the contours of c
    x, y, w, h = cv2.boundingRect(c)
    xy.append([x, y, w, h])
    cv2.imwrite(i,mask)
    

In [ ]:
import pickle
with open("test.txt", "wb") as fp:
    pickle.dump(xy, fp)

In [ ]:
# with open("test.txt", "rb") as fp:
#     b = pickle.load(fp)
# print(b)

In [ ]:
!mkdir masks
!mv * masks

In [ ]:
!apt install zip


In [ ]:
!zip -r masks.zip masks 


In [ ]:
!rm masks -r